In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import pickle
import time
import random
import unicodedata

In [2]:
# !pip install selenium

In [3]:
options =  Options()
    # options.set_headless(True)
driver = webdriver.Firefox(options=options)

In [ ]:
driver.get("https://bcinform.ru/spb/liteynyy-dvor.html")

In [ ]:
page  = driver.page_source 
soup = BeautifulSoup(page, 'html5lib')

In [ ]:
soup.find("div",{"class":"name"}).text.replace("  ","").replace("\n","")

In [ ]:
re.findall(r"\d+",soup.find("div",{"class":"tax-number"}).text.replace("  ",""))[0]

In [ ]:
ininfo={}
for i in soup.find("div",{"class":"services"}).find_all("div",{"class":"single-service tooltipped"}):
    ininfo[i.span.text]=i.div.text

outinfo=[]
for i in soup.find("div",{"class":"services"}).find_all("div",{"class":re.compile("missing")}):
    outinfo+=[i.span.text]
outinfo

In [ ]:
ofs=[]
for o in soup.find_all("tr",{"class":"object-id"}):
    ofiss={}
    ofiss["name"]=o.find_all("td")[1].text
    
    ofiss["floor"]=o.find_all("td")[2].text.replace(" ","").replace("\n","")
    ofiss["meters"]=o.find_all("td")[3].text.replace(" ","").replace("\n","")
    ofiss["rub/metr"]=o.find_all("td")[4].text.replace(" ","").replace("\n","")
    ofiss["cost"]=o.find_all("td")[5].text.replace(" ","").replace("\n","")
    
    ofs.append(ofiss)

In [ ]:
ofs

In [4]:
def pars(url):
    driver.get(url)
    item={}
    page  = driver.page_source 
    soup = BeautifulSoup(page, 'html5lib')
    
    if soup.h1.text=="Помещение сдано":
        item["name"] = "Здано"
        item["url"]=url
        return item

    item["name"]=soup.find_all("span",{"itemprop":"name"})[-1].text
    item["phone"]=soup.find("div",{"class":"phone"})["data-phone"]
    item["call_to"]=soup.find("div",{"class":"name"}).text.replace("  ","").replace("\n","")
    item["adress"]=soup.find("div",{"class":"street"}).text.replace("  ","").replace("\n","").replace("\t","")
    item["url"]=url
    item["level"]=soup.find("span",{"class":"white-text badge"}).text
    item["description"] = soup.find("div",{"itemprop":"description"}).text.replace("  ","")
    item["call-time"] = soup.find("div",{"class":"time"}).text.replace("  ","").replace("\n","")

    if soup.find("div",{"class":"services"}) is not None:
        item["ininfo"]={}
        for i in soup.find("div",{"class":"services"}).find_all("div",{"class":"single-service tooltipped"}):
            item["ininfo"][i.span.text]=i.div.text

        item["outinfo"]=[]
        for i in soup.find("div",{"class":"services"}).find_all("div",{"class":re.compile("missing")}):
            item["outinfo"]+=[i.span.text]

    ofs=[]
    for o in soup.find_all("tr",{"class":"object-id"}):
        ofiss={}
        try:
            ofiss["name"]=o.find_all("td")[1].text
            ofiss["floor"]=o.find_all("td")[2].text.replace(" ","").replace("\n","")
            ofiss["meters"]=o.find_all("td")[3].text.replace(" ","").replace("\n","")
            ofiss["rub/metr"]=o.find_all("td")[4].text.replace(" ","").replace("\n","")
            ofiss["cost"]=o.find_all("td")[5].text.replace(" ","").replace("\n","")
        except:
            pass
        ofs.append(ofiss)
        item["etag"]=ofiss["floor"]
        
    item["ofis"]=ofs
    
    if soup.find("div",{"class":"tax-number"}) is not None:
        item["tax-number"] = re.findall(r"\d+",soup.find("div",{"class":"tax-number"}).text.replace("  ",""))
        if len(item["tax-number"] )>0:
            item["tax-number"] =item["tax-number"][0]

   
    return item

In [5]:
pars("https://bcinform.ru/spb/start-servis.html")

{'name': 'Старт-сервис',
 'phone': '+7 (921) 326 05 33',
 'call_to': 'Максим Викторович(отдел аренды)',
 'adress': 'Салова ул. 27 / ад',
 'url': 'https://bcinform.ru/spb/start-servis.html',
 'level': 'C',
 'description': '\n\nБизнес-центр «Старт-сервис» представляет собой деловой комплекс с современным оснащением и уютным интерьером. В шаговой доступности находится станция метро «Волковская». В БЦ возможна аренда складских и торговых помещений. Удачное размещение БЦ во Фрунзенском районе, близость остановок общественного транспорта обеспечат высокую мобильность.\xa0\nХарактеристика БЦ «Старт-сервис»\nБизнес-центр «Старт-сервис» – это объект, включающий в себя офисные помещения. Объект оснащен необходимыми инженерными системами и коммуникациями. В бизнес-центре поддерживается высокий уровень безопасности, осуществляется пропускной режим. Для удобства арендаторов помещений, их гостей и деловых партнеров БЦ «Старт-сервис» имеет собственный паркинг.\nАренда офиса в бизнес-центре «Старт-сер

In [6]:
ofises=[]
parsedurls=[]

In [16]:
# parsedurls=[]
# with open("ofises","rb") as f:
#     ofises = pickle.load(f)
    
# with open("urls2","rb") as f:
#     urls = pickle.load(f)
for o in ofises:
    parsedurls.append(o["url"])

In [17]:
def bash(bashCommand):
    import subprocess
    process = subprocess.Popen(bashCommand, stdout=subprocess.PIPE,shell=True)
    output, error = process.communicate()
    output = output.decode("utf-8")[:-1]
    return output


In [19]:
pattern = "\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}"
ips=["1"]
ip2="1"

for i,u in enumerate(tqdm(list(set(urls)))):
    if u in parsedurls:
        continue
        
    try:
        ofises.append(pars(u))
    except Exception as e:
        if driver.page_source.find("Too many requests!")==-1:
            print(e)
            continue
            
        driver.close()
        while ip2 in ips:
            s = bash("windscribe connect US-W")
            ip2=re.compile(pattern).findall(s)[1]
        ips.append(ip2)
        
        driver = webdriver.Firefox(options=options)
        print("New ip: "+ip2)
        time.sleep(2)  
        try:
            ofises.append(pars(u))
        except:
            time.sleep(30)
            continue
        
        
    parsedurls.append("https://bcinform.ru"+u)
    if i%5==1:
        with open("bc_ofises","wb") as f:
            pickle.dump(ofises,f)


'NoneType' object has no attribute 'text'
New ip: 23.94.74.103
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'



In [20]:
import json

with open('output.jsonl', 'a') as f:
    for entry in ofises:
        json.dump(entry, f)
        f.write('\n')
with open('output.jsonlines', 'a') as f:
    for entry in ofises:
        f.write(json.dumps(entry)+'\n')

In [21]:
with open("ofises","wb") as f:
    pickle.dump(ofises,f)

In [ ]:
regex_num = re.compile('https://bcinform.ru/spb/.*?html') 
urls=[]
for i in tqdm(range(1,65)):
    driver.get(f"https://bcinform.ru/spb/biznes-centry/page-{i}/")
    page  = driver.page_source 
    ur= list(set(regex_num.findall(page)))
    if len(ur)==0:
        time.sleep(random.randint(50,150))
    else:
        urls+=ur
    time.sleep(random.randint(5,15))

In [11]:
with open("bc_urls","rb") as f:
    urls = pickle.load(f)

In [12]:
with open("bc_ofises","rb") as f:
    ofises = pickle.load(f)

In [14]:
len(ofises)

571

In [15]:
ofises[0]

{'name': 'Московское шоссе 179',
 'phone': '+7 (921) 325 79 85',
 'call_to': 'Алексей Иванович(отдел аренды)',
 'adress': 'Московское ш. 179 / 2',
 'url': 'https://bcinform.ru/spb/moskovskoe-shoss-179.html',
 'level': 'B',
 'description': '\n\n\n',
 'call-time': 'Круглосуточно',
 'ininfo': {'Парковка': 'Всего  100 парковочных мест',
  'Режим работы': 'Круглосуточно',
  'УСН или НДС': 'НДС включен в стоимость',
  'Охрана': 'Строго по документам'},
 'outinfo': ['Клининг', 'Банкоматы', 'Интернет и телефония'],
 'ofis': [{'name': '\n                                                            \n                                                                Офис 34.2 м2                                                                \n                                                            ',
   'floor': '4',
   'meters': '550',
   'rub/metr': '18810'},
  {'name': '\n                                                            \n                                                            